In [ ]:
pip install pydub librosa numpy torchaudio speechbrain
pip install webrtcvad

import webrtcvad
from pydub import AudioSegment
from pydub.playback import play

input_file = r'./Audio Files/Her/Her.mp4'
output_file = r'./Audio Files/Her/HerOutput.wav'

def convert_to_wav(input_file, output_file):
    audio = AudioSegment.from_file(
        input_file,
        format = 'mp4',
        )
    audio.export(output_file, format='wav')


convert_to_wav(input_file, output_file)



Create sample files

start time in ms is 1511720 and end time in ms is 1512807


<_io.BufferedRandom name='./Data/Training_Samples/sample1.wav'>

# For Analyzing Speech (e.g., Phoneme Conversion):

    Lower hop_length:
        Provides better time resolution, allowing the model to capture finer details, especially crucial for accurately segmenting phonemes and detecting quick transitions between them.
        More frames = higher granularity in time.
    Higher n_fft:
        Provides better frequency resolution, allowing the model to capture finer frequency details, which are important for distinguishing between different phonemes based on their spectral features.
        More frequency bins = more precise frequency representation.

For Generating Speech (e.g., Speech Synthesis):

    Higher hop_length:
        Reduces the number of frames, which decreases computational cost. It also provides less granular time resolution, but this is often acceptable for generating smoother speech and avoiding overfitting on small time variations.
    Lower n_fft:
        Provides coarser frequency resolution, which can help model more general frequency patterns, reducing computational complexity while still maintaining enough frequency detail for natural speech synthesis.

Summary:

    For analysis: Lower hop_length and higher n_fft (better time and frequency resolution).
    For generation: Higher hop_length and lower n_fft (better computational efficiency, but with slightly reduced resolution in both time and frequency).

Converting WAV --> signal (2D Numpy array)

float32 (23969,) [ 0.00146869  0.00227872  0.00214889 ... -0.00629799 -0.00731505
  0.        ]
torch.Size([128, 94])


Converting tensors back into WAV files

torch.Size([80, 47]) 
n_fft = 1024  # Window length
hop_length = 512 # Hop length
n_mels = 80


torch.Size([128, 94])
n_fft = 2048  # Window length
hop_length = 256 # Hop length
n_mels = 128

Number of Frames=⌊(Signal Length−n_fft)​/hop_length⌋+1

In [ ]:
librosa.display.waveshow(y=signal, sr=sr)
plt.title('Signal')
plt.xlabel('Time (samples)')
plt.ylabel('Amplitude')
plt.show()

plt.figure(figsize=(10, 4))
img = librosa.display.specshow(
    log_mel_spectrogram, 
    sr=sr, 
    hop_length=hop_length, 
    x_axis='time', 
    y_axis='mel', 
    fmax=8000, 
    cmap='magma'
)
plt.colorbar(img, label='DB')  # Add a label to the colorbar
plt.title('Log Mel Spectrogram')
plt.xlabel('Time (s)')  # Label for the x-axis
plt.ylabel('Frequency (Hz)')  # Label for the y-axis
plt.grid(True, linestyle='--', alpha=0.6)  # Optional: add grid for better readability
plt.tight_layout()
plt.show()



# Reverse Process
mel_spectrogram_reversed = librosa.db_to_power(log_mel_spectrogram, ref=np.max(mel_spectrogram))
linear_spectrogram = librosa.feature.inverse.mel_to_stft(mel_spectrogram_reversed, sr=sr, n_fft=n_fft)

# Use Griffin-Lim to reconstruct the signal
reconstructed_signal = librosa.griffinlim(linear_spectrogram, hop_length=hop_length, n_iter=64)

# Save reconstructed audio
sf.write('reconstructed_audio.wav', reconstructed_signal, sr)



output_file = 'output.wav'  # Specify the output file name

# Ensure the signal is scaled to the appropriate range for WAV files
# WAV files typically expect 16-bit PCM, so we scale the signal to int16
scaled_signal = (signal * 32767).astype(np.int16)

# Write the WAV file
write(output_file, sr, scaled_signal)

print(f"WAV file written to {output_file}")

In [8]:
import utils
import soundfile as sf


import re
import utils
import librosa
import librosa.display
import numpy as np
import torch
import soundfile as sf 

def read_srt_file():
    file_path = utils.get_srt()
    if not file_path:
        raise FileNotFoundError("SRT file path not found in the configuration.")
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

input_file = read_srt_file()

# Regular expression
pattern = r'(\d{3}|\d{4})\s+(\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3})\s+([\s\S]*?)(?=</font>|</i></font>)'

# Find all matches
matches = re.findall(pattern, input_file, re.DOTALL)

data_dict = {match[0]: {"Timestamp": match[1], "Text": match[2]} for match in matches}

vocab = []

#cleaning transcription by moving values from SRT file
for key, value in data_dict.items():
    value['Text'] = (
        value['Text']
        .replace('\n', ' ')
        .replace('/', '')
        .replace('<b><font color="#ffff00">', '')
        .replace('<b><font color="#ffff00"><i>', '')
        .replace('</i></font></b>', '')
        .replace('<i>','')
        .lower()
    )
    value['Text'] = [
        '<SPACE>' if token.isspace() else token # convert spaces into <SPACE>
        for token in re.findall(r'\b\w+\'?\w*|[.,!?]|\s', value['Text']) #convert text into list with words, <SPACE>, and .,!?
    ]
    vocab += value['Text']
    value['Timestamp'] = [utils.t2ms(split) for split in value['Timestamp'].split('-->')]

print(data_dict['167']['Timestamp'])


[753080, 755287]


In [9]:
whole_audio = utils.get_whole_audio()
audio = librosa.load(whole_audio)

for key, value in data_dict.items():
    # Extract the start and end times from the "Timestamp" key
    start_t, end_t = value["Timestamp"]
    cut_audio = audio[start_t:end_t]
    output_file = f"./Data/Training_Samples/{key}.wav"
    sf.write(output_file, cut_audio, 22050)
    break

KeyboardInterrupt: 

In [11]:
file = '123.wav'
trunc_file = file.split('.wav')
print(trunc_file)

['123', '']
